In [2]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
df = pd.read_csv('results.csv')
df.head()

25k  age               name  division    10k gender   half  official bib  \
0  49.87   28   Cassidy, Josh R.         9  18.18      M  40.93     90.90  W1   
1  77.27   30      Korir, Wesley         5   30.9      M   64.9    132.50   1   
2  77.23   23     Desisa, Lelisa         1   30.9      M  64.92    130.37   2   
3   50.5   32  Fearnley, Kurt H.         5  18.73      M     42     88.43  W2   
4  48.75   39     Hokinoue, Kota         3  18.18      M  40.57     87.22  W3   

   ctz  ... overall  pace  state    30k     5k genderdiv    20k     35k  \
0  NaN  ...       9  3.47     ON  62.07    8.9         9   38.8   74.73   
1  NaN  ...       5  5.07    NaN  92.97   15.9         5  61.52  108.78   
2  NaN  ...       1  4.98    NaN  92.72  15.93         1  61.53  108.68   
3  NaN  ...       5  3.38    NaN  61.35   8.98         5  39.88      73   
4  NaN  ...       3  3.33    NaN  59.92   8.92         3  38.55   71.68   

       city     40k  
0   Toronto   85.55  
1     Kenya  124.77  
2      Ambo  123.78  
3  Hamilton   83.43  
4    Iizuka   81.88  

[5 rows x 21 columns]

In [ ]:

df = pd.read_csv(results.csv)

# Make sure the number of rows divides evenly into four samples.
rows = df.shape[0] - df.shape[0] % 4
df = df.iloc[:rows, :]

# Break into a set of features and a variable for the known outcome.
X = df.iloc[:, :13]
y = df.iloc[:, 13]

# Replace some random string values.
X = X.replace(to_replace='?', value=0)

# Binarize y so that 1 means heart disease diagnosis and 0 means no diagnosis.
y = np.where(y > 0, 0, 1)

# Normalize
X_norm = normalize(X)

# Data frame to store features and predicted cluster memberships.
ypred = pd.DataFrame()

# Create the two-feature PCA for graphing purposes.
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_norm)

# Split the data into four equally-sized samples. First we break it in half:
X_half1, X_half2, X_pcahalf1, X_pcahalf2 = train_test_split(
    X_norm,
    X_pca,
    test_size=0.5,
    random_state=42)

# Then we halve the halves.
X1, X2, X_pca1, X_pca2 = train_test_split(
    X_half1,
    X_pcahalf1,
    test_size=0.5,
    random_state=42)
X3, X4, X_pca3, X_pca4 = train_test_split(
    X_half2,
    X_pcahalf2,
    test_size=0.5,
    random_state=42)

# Pass a list of tuples and a counter that increments each time we go
# through the loop. The tuples are the data to be used by k-means,
# and the PCA-derived features for graphing. We use k-means to fit a
# model to the data, then store the predicted values and the two-feature
# PCA solution in the data frame.
for counter, data in enumerate([
    (X1, X_pca1),
    (X2, X_pca2),
    (X3, X_pca3),
    (X4, X_pca4)]):
    
    # Put the features into ypred.
    ypred['pca_f1' + '_sample' + str(counter)] = data[1][:, 0]
    ypred['pca_f2' + '_sample' + str(counter)] = data[1][:, 1]
    
    # Generate cluster predictions and store them for clusters 2 to 4.
    for nclust in range(2, 5):
        pred = KMeans(n_clusters=nclust, random_state=42).fit_predict(data[0])
        ypred['clust' + str(nclust) + '_sample' + str(counter)] = pred